hello 


In [1]:
from IncrementalTrainingApproach.iCaRL import iCaRLmodel

In [2]:
from models import get_feature_extractor
feature_extractor = get_feature_extractor('cct7_h')
trainer = iCaRLmodel(6,feature_extractor,32,10,2000,10,1)

Files already downloaded and verified
Files already downloaded and verified


Hello
Tokenizer(
  (conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ReLU()
      (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
  )
  (flattener): Flatten(start_dim=2, end_dim=3)
)


In [3]:
trainer.beforeTrain()



the size of train set is (3000, 32, 32, 3)
the size of train label is (3000,)
the size of test set is (600, 32, 32, 3)
the size of test label is (600,)


In [4]:
import os,wandb
os.environ['WANDB_MODE'] = 'offline'
wandb.init()
trainer.train()

wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
Training:   0%|          | 0/94 [00:00<?, ?it/s]


epoch:1,accuracy:16.667
epoch:2,accuracy:16.667
epoch:3,accuracy:16.667
epoch:4,accuracy:16.667
epoch:5,accuracy:16.667
epoch:6,accuracy:16.667


KeyboardInterrupt: 

In [7]:
trainer.afterTrain(1)

construct class 0 examplar:the size of exemplar :200
construct class 1 examplar:the size of exemplar :200
construct class 2 examplar:the size of exemplar :200
construct class 3 examplar:the size of exemplar :200
construct class 4 examplar:the size of exemplar :200
construct class 5 examplar:the size of exemplar :200
construct class 6 examplar:the size of exemplar :200
construct class 7 examplar:the size of exemplar :200
construct class 8 examplar:the size of exemplar :200
construct class 9 examplar:the size of exemplar :200
compute the class mean of 0
compute the class mean of 1
compute the class mean of 2
compute the class mean of 3
compute the class mean of 4
compute the class mean of 5
compute the class mean of 6
compute the class mean of 7
compute the class mean of 8
compute the class mean of 9
compute NMS
NMS accuracy：10.0


Testing: 100%|██████████| 32/32 [00:06<00:00,  5.26it/s]
